In [10]:
#import pyarrow.parquet as pq

import pandas as pd
import numpy as np


In [11]:
#To read the parquet file using pyarrow
#trips = pq.read_table('trips.parquet')
#trips = trips.to_pandas()


#Reading the parquet file using pandas
trips = pd.read_parquet('C:/Users/Arpita/Downloads/yellow_tripdata_2022-01.parquet',engine="auto",columns=None)
#deriving a new column of time taken for the trip
trips["total time taken"]=trips["tpep_dropoff_datetime"]-trips["tpep_pickup_datetime"]
trips.min()


C:\Users\Arpita\AppData\Local\Temp\ipykernel_8128\2454422005.py:10: FutureWarning: The default value of numeric_only in DataFrame.min is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  trips.min()


VendorID                                   1
tpep_pickup_datetime     2008-12-31 22:23:09
tpep_dropoff_datetime    2008-12-31 23:06:56
passenger_count                          0.0
trip_distance                            0.0
RatecodeID                               1.0
PULocationID                               1
DOLocationID                               1
payment_type                               0
fare_amount                           -480.0
extra                                   -4.5
mta_tax                                 -0.5
tip_amount                           -125.22
tolls_amount                           -31.4
improvement_surcharge                   -0.3
total_amount                          -480.3
congestion_surcharge                    -2.5
airport_fee                            -1.25
total time taken           -3 days +14:37:36
dtype: object

In [12]:
#Taking the subset of first 70 rowws of the given data
dfh=trips.head(70)
#print(dfh)

In [13]:
#checking for the maximum value of each column
dfh.max()

VendorID                                   2
tpep_pickup_datetime     2022-01-01 18:04:06
tpep_dropoff_datetime    2022-01-01 18:40:52
passenger_count                          6.0
trip_distance                           19.5
RatecodeID                               2.0
store_and_fwd_flag                         Y
PULocationID                             263
DOLocationID                             263
payment_type                               4
fare_amount                             52.0
extra                                   3.75
mta_tax                                  0.5
tip_amount                             15.75
tolls_amount                            6.55
improvement_surcharge                    0.3
total_amount                           78.85
congestion_surcharge                     2.5
airport_fee                             1.25
total time taken             0 days 00:37:32
dtype: object

In [14]:
#reading the column names from the file
print(trips.head(0))

Empty DataFrame
Columns: [VendorID, tpep_pickup_datetime, tpep_dropoff_datetime, passenger_count, trip_distance, RatecodeID, store_and_fwd_flag, PULocationID, DOLocationID, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount, congestion_surcharge, airport_fee, total time taken]
Index: []


In [15]:
#checking for the enries with null values
print(trips.info(null_counts=True))

C:\Users\Arpita\AppData\Local\Temp\ipykernel_8128\81961390.py:2: FutureWarning: null_counts is deprecated. Use show_counts instead
  print(trips.info(null_counts=True))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2463931 entries, 0 to 2463930
Data columns (total 20 columns):
 #   Column                 Non-Null Count    Dtype          
---  ------                 --------------    -----          
 0   VendorID               2463931 non-null  int64          
 1   tpep_pickup_datetime   2463931 non-null  datetime64[ns] 
 2   tpep_dropoff_datetime  2463931 non-null  datetime64[ns] 
 3   passenger_count        2392428 non-null  float64        
 4   trip_distance          2463931 non-null  float64        
 5   RatecodeID             2392428 non-null  float64        
 6   store_and_fwd_flag     2392428 non-null  object         
 7   PULocationID           2463931 non-null  int64          
 8   DOLocationID           2463931 non-null  int64          
 9   payment_type           2463931 non-null  int64          
 10  fare_amount            2463931 non-null  float64        
 11  extra                  2463931 non-null  float64        
 12  mta_tax       

In [16]:
#removing the null values
trips.dropna(how="any",inplace = True)

Plotting the graphs:

Visualization-1: A distance vs time glyph

In [17]:
from datetime import datetime, timedelta

from bokeh.models import DatetimeTickFormatter, NumeralTickFormatter,Spinner,Div,BoxAnnotation
from bokeh.plotting import figure, show, output_notebook,output_file,curdoc
from bokeh.layouts import layout
output_notebook


dist=dfh["trip_distance"]*1000
time=dfh["total time taken"].dt.total_seconds()/60
#radii=dist/20000

curdoc().theme = "light_minimal"

colors1 = ["#%02x%04x%02x" % (255, int(round(value * 255 / 100)), 255) for value in dist]
colors2 = ["#%04x%02x%02x" % (255, int(round(value * 255 / 100)), 255) for value in time]


graph=figure(title="Average speed of taxi for each trip",
             sizing_mode="stretch_width",
             height=500,
             x_axis_label="Time (in minutes)",
             y_axis_label="Distance covered (in meters)")
#graph.line(x=time,y=dist,legend_label="avg speed",line_width=0.4,color="red")
bar=graph.vbar(x=time,top=dist,fill_color=colors2,width=0.05,bottom=0)
circle=graph.circle(x=time,y=dist,fill_color=colors1,size =20)
circle2=graph.circle(x=time,y=dist,legend_label="avg speed",color="navy",size =2)

# add box annotations
low_box = BoxAnnotation(top=5000, fill_alpha=0.2, fill_color="green")
mid_box1 = BoxAnnotation(bottom=5000, top=10000, fill_alpha=0.2, fill_color="olive")
mid_box2 = BoxAnnotation(bottom=10000, top=15000, fill_alpha=0.2, fill_color="blue")
high_box = BoxAnnotation(bottom=15000, fill_alpha=0.2, fill_color="navy")

# add boxes to existing figure
graph.add_layout(low_box)
graph.add_layout(mid_box1)
graph.add_layout(mid_box2)
graph.add_layout(high_box)


# set up textarea (div)
#div = Div( 
# text="""
#          <p>Select the circle's size using this control element:</p>
 #         """,
  #  width=200,
   # height=30,
#)

# set up spinner
#spinner = Spinner(
 #   title="Circle size",
  #  low=10,
   # high=500,
    #step=10,
    #value=circle.glyph.size,
    #width=200,
#)
#spinner.js_link("value", circle.glyph, "size")

#layout = layout([[div, spinner],[graph]])
#show(layout)

show(graph)